In [9]:
%cd ../..
%load_ext autoreload
%autoreload 2
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams.update({'font.size': 20})
from pipeline import lab, get_schema_name, experiment, foraging_model, ephys, foraging_analysis, histology, ccf, report
from pipeline.plot import unit_psth
from pipeline.plot.foraging_model_plot import plot_session_model_comparison, plot_session_fitted_choice

import os
dj.config['query_cache'] = os.path.expanduser('~/dj_query_cache')
# access the active connection object for the tables
conn = dj.conn() # if queries co-located with tables
# activate query caching for a namespace called 'main'
conn.set_query_cache(query_cache='main')

/home
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Total foraging units

# Unit quality control

In [2]:
q_unit_foraging = ephys.Unit & foraging_analysis.SessionTaskProtocol
total = len(q_unit_foraging)
total

40845

In [6]:
qc_criteria = {
    'before': 'presence_ratio > 0.9 '
              'AND amplitude_cutoff < 0.1 '
              'AND isi_violation < 0.5 '
              'AND unit_amp > 70',
    'minimal': 'unit_amp > 70 '
               'AND avg_firing_rate > 0.1 '
               'AND presence_ratio > 0.9 '
               'AND isi_violation < 0.1 '
               'AND amplitude_cutoff < 0.15',
    'Medulla': 'unit_amp > 150 '
               'AND avg_firing_rate > 0.2 '
               'AND presence_ratio > 0.9 '
               'AND isi_violation < 10 '
               'AND amplitude_cutoff < 0.15',
    'ALM': 'unit_amp > 100 '
           'AND avg_firing_rate > 0.2 '
           'AND presence_ratio > 0.95 '
           'AND isi_violation < 0.1 '
           'AND amplitude_cutoff < 0.1',
    'Midbrain': 'unit_amp > 100 '
                'AND avg_firing_rate > 0.1 '
                'AND presence_ratio > 0.9 '
                'AND isi_violation < 1 '
                'AND amplitude_cutoff < 0.08',
    'Thalamus': 'unit_amp > 90 '
                'AND avg_firing_rate > 0.1 '
                'AND presence_ratio > 0.9 '
                'AND isi_violation < 0.05 '
                'AND amplitude_cutoff < 0.08',
    'Striatum': 'unit_amp > 70 '
                'AND avg_firing_rate > 0.1 '
                'AND presence_ratio > 0.9 '
                'AND isi_violation < 0.5 '
                'AND amplitude_cutoff < 0.1'
}

In [7]:
for qc in qc_criteria:
    qced = q_unit_foraging * ephys.ClusterMetric * ephys.UnitStat & qc_criteria[qc]
    print(f'{qc}: {len(qced)}, {len(qced)/total:.2%}')

before: 11682, 28.60%
minimal: 9142, 22.38%
Medulla: 4601, 11.26%
ALM: 5900, 14.44%
Midbrain: 8099, 19.83%
Thalamus: 6099, 14.93%
Striatum: 11620, 28.45%


In [14]:
(ephys.ProbeInsertion * experiment.Session & 'username="hh"' & histology.ElectrodeCCFPosition())

subject_id institution 6 digit animal ID,session session number,insertion_number,probe unique identifier for this model of probe (e.g. part number),probe_type e.g. neuropixels_1.0,electrode_config_name user friendly name,session_date,session_time,username,rig
473360,49,1,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-12,16:52:05,HH,Ephys-Han
473360,49,2,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-12,16:52:05,HH,Ephys-Han
473360,50,1,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-13,16:33:03,HH,Ephys-Han
473360,50,2,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-13,16:33:03,HH,Ephys-Han
473360,51,1,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-14,18:08:26,HH,Ephys-Han
473360,51,2,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-14,18:08:26,HH,Ephys-Han
473360,51,3,19122505354,neuropixels 2.0 - SS,1-384,2021-08-14,18:08:26,HH,Ephys-Han
473360,52,1,18005108622,neuropixels 1.0 - 3B,1-384,2021-08-15,18:19:35,HH,Ephys-Han
473360,52,2,18005123491,neuropixels 1.0 - 3B,1-384,2021-08-15,18:19:35,HH,Ephys-Han
473360,52,3,18005117122,neuropixels 1.0 - 3B,1-384,2021-08-15,18:19:35,HH,Ephys-Han


# Area of interest

## Look up table

In [ ]:
# check session stats for a region
# fetch data from brain region
region_ann_lut = {
    # premotor
    'ALM': ["Secondary motor area%"],
    
    # isocortex, PFC
    'PL': ["Prelimbic%"],
    'ACA': ["Anterior cingulate area%"],
    'ILA': ["Infralimbic%"],
    'ORB': ['%orbital%'],
    # 'FRP': '%frontal%',
    'RSP': ["Retrosplenial area%"],
    
    # thalamus
    'VM': ['Ventral medial%'],
    'MD': ['Mediodorsal%'],
    'VPM': ['Ventral posteromedial%'],
    'HY': ['Hypothalamus', 'Zona %'],
    
    # striatum
    'LSN': ["Lateral septal nucleus%"],
    'STRd': ["Caudoputamen%"],
    'STRv': ["Nucleus accumbens%", "Fundus%"],
    'striatum': ["striatum%"],
    
    # Pallidum
    'PALv': ["Substantia innominata%", "Magnocellular%"],
    
    # Olfactory
    'OLF': ["%olfactory%"],
}

In [13]:
ccf.AreaOfInterest.CCFBrainRegionIncluded.proj(annotation='region_name').fetch(format='frame')

annotation
area_of_interest id                                                   
OLF              0                            Accessory olfactory bulb
                 1          Accessory olfactory bulb, glomerular layer
                 2            Accessory olfactory bulb, granular layer
                 3              Accessory olfactory bulb, mitral layer
                 4                           accessory olfactory tract
ACA              0                             Anterior cingulate area
                 1                Anterior cingulate area, dorsal part
                 2       Anterior cingulate area, dorsal part, layer 1
                 3     Anterior cingulate area, dorsal part, layer 2/3
                 4       Anterior cingulate area, dorsal part, layer 5
                 5      Anterior cingulate area, dorsal part, layer 6a
                 6      Anterior cingulate area, dorsal part, layer 6b
                 7                    Anterior cingulate area, layer 1
                 8                  Anterior cingulate area, layer 2/3
                 9                    Anterior cingulate area, layer 5
                 10                  Anterior cingulate area, layer 6a
                 11                  Anterior cingulate area, layer 6b
                 12              Anterior cingulate area, ventral part
                 13          Anterior cingulate area, ventral part, 6a
                 14          Anterior cingulate area, ventral part, 6b
                 15     Anterior cingulate area, ventral part, layer 1
                 16   Anterior cingulate area, ventral part, layer 2/3
                 17     Anterior cingulate area, ventral part, layer 5
OLF              5                 anterior commissure, olfactory limb
                 6                          Anterior olfactory nucleus
                 7             Anterior olfactory nucleus, dorsal part
                 8           Anterior olfactory nucleus, external part
                 9            Anterior olfactory nucleus, lateral part
                 10                Anterior olfactory nucleus, layer 1
                 11                Anterior olfactory nucleus, layer 2
                 12            Anterior olfactory nucleus, medial part
                 13    Anterior olfactory nucleus, posteroventral part
                 14       Bed nucleus of the accessory olfactory tract
STRd             0                                        Caudoputamen
STRv             0                                  Fundus of striatum
HY               0                                        Hypothalamus
ILA              0                                    Infralimbic area
                 1                           Infralimbic area, layer 1
                 2                           Infralimbic area, layer 2
                 3                         Infralimbic area, layer 2/3
                 4                           Infralimbic area, layer 5
                 5                          Infralimbic area, layer 6a
                 6                          Infralimbic area, layer 6b
OLF              15                      lateral olfactory tract, body
                 16                   lateral olfactory tract, general
LSN              0                              Lateral septal nucleus
                 1   Lateral septal nucleus, caudal (caudodorsal) part
                 2   Lateral septal nucleus, rostral (rostroventral...
                 3                Lateral septal nucleus, ventral part
PALv             0                               Magnocellular nucleus
                 1                     Magnocellular reticular nucleus
OLF              17                                Main olfactory bulb
                 18              Main olfactory bulb, glomerular layer
                 19                 Main olfactory bulb, granule layer
                 20         Main olfactory bulb, inner plexiform layer
             

In [6]:
units_aoi = (ephys.Unit & report.UnitLevelForagingEphysReportAllInOne.key_source) * histology.ElectrodeCCFPosition.ElectrodePosition * ccf.CCFAnnotation * ccf.AreaOfInterest.CCFBrainRegionIncluded.proj(annotation='region_name')
units_aoi = units_aoi * lab.WaterRestriction.proj('water_restriction_number')

In [10]:
pd.set_option('display.max_rows', 1000)

## Number of units per area of interest, grouped by session

In [12]:
dj.U('area_of_interest', 'water_restriction_number', 'session').aggr(units_aoi, count='COUNT(*)').fetch(format='frame')

count
area_of_interest water_restriction_number session      
ACA              HH08                     49         11
                                          50         37
                                          51          2
                                          52         52
                 HH09                     47         13
                 HH13                     34         32
                                          36         66
                                          37         36
                                          43         48
                 HH16                     17         36
                                          18         36
ALM              HH08                     49          1
                                          50         13
                                          51         73
                                          52         87
                 HH09                     47          3
                 HH13                     33         31
                                          34        112
                                          36        171
                                          37         51
                 HH16                     17         79
                                          18         16
HY               HH08                     49          1
                                          52          1
                 HH09                     57         21
                 HH13                     36         12
                                          42         81
                                          43        163
                                          45         10
ILA              HH08                     49         10
                                          50         19
                 HH09                     47          4
                 HH13                     36          1
                 HH16                     17         44
                                          18         37
LSN              HH08                     50        159
                                          51         25
                                          52         91
                 HH13                     33         77
                                          34        121
                                          36         83
                                          37         85
                                          42         98
                 HH16                     18         24
MD               HH09                     60         45
OLF              HH09                     47          2
                 HH13                     33         45
                                          34         28
                                          36         32
                                          45          5
                 HH16                     17         97
ORB              HH09                     47          5
                 HH13                     33         74
                                          34         50
                                          36         61
                 HH16                     17         72
PALv             HH09                     57         20
                 HH13                     34         12
                                          37          1
                                          45         21
                 HH16                     17          5
PL               HH08                     49         31
                                          50         49
                                          51          6
                                          52         44
                 HH09                     47         29
                 HH13                     36          6
                                          37          2
                                          43         12
                 HH16                     17        121
                  